In [1]:
"""
Pedro Tobarra 20210824:
Modificacion del notebook '20210616 seguro medico prophet ide.ipynb' para meter el codigo en una funcion de
python que luego pueda ser implementada en un fichero '.py' para su integracion con el backend del PFM
"""

"\nPedro Tobarra 20210824:\nModificacion del notebook '20210616 seguro medico prophet ide.ipynb' para meter el codigo en una funcion de\npython que luego pueda ser implementada en un fichero '.py' para su integracion con el backend del PFM\n"

In [2]:
import pandas
from pandas import read_excel, date_range, DatetimeIndex, DataFrame, to_numeric, concat, to_datetime
from pandas.plotting import lag_plot, autocorrelation_plot
from pandas.tseries.offsets import DateOffset

import numpy
from numpy import logical_not

import statistics
import dateutil.relativedelta

import fbprophet
from fbprophet import Prophet

import calendar
from calendar import monthrange

from datetime import datetime, timedelta
from matplotlib import pyplot

from sklearn.metrics import mean_absolute_error, mean_squared_error

from math import sqrt

In [3]:
# cambiamos el valor de estos 4 parametros para que nos muestre dataframes sin truncarlos
pandas.set_option('display.max_rows', 12)
pandas.set_option('display.max_columns', None)
pandas.set_option('display.width', None)
pandas.set_option('display.max_colwidth', None)

In [4]:
# Load data using read_excel
transacciones_df = read_excel('20210513 mmelero (249236).xlsx', sheet_name='Hoja1')
transacciones_df

,Fecha transacción,Importe,ID Categoría,Nombre categoría,Nivel categoría,iD categoría padre,Nombre categoría padre,Proveedor,Marca,Sector,Financiero,Transferencia,Unnamed: 12,Unnamed: 13,ID,BALANCE,BALANCE_DATE
0,2020-07-24,-1.20,84.0,parking_84,2.0,5.0,vehiculos_5,NaN,NaN,NaN,False,False,NaN,NaN,249236.0,1478.98,2020-07-26
1,2020-07-23,-4.50,117.0,regalos_celebraciones_117,2.0,8.0,gastos_personales_8,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaT
2,2020-07-23,-13.53,70.0,supermercados_70,2.0,4.0,alimentacion_4,SPAR,SPAR,Supermercados,False,False,NaN,NaN,NaN,NaN,NaT
3,2020-07-22,-51.40,222.0,adsl_222,3.0,521.0,Comunicaciones y TV,Jazz Telecom,Jazztel,Compañías telecomunicaciones,False,False,NaN,NaN,NaN,NaN,NaT
4,2020-07-22,-11.00,70.0,supermercados_70,2.0,4.0,alimentacion_4,Mercadona,Mercadona,Supermercados,False,False,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482,2016-09-05,1533.02,18.0,retribucion_liquida_18,3.0,576.0,Nómina,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaT
1483,2016-09-05,-617.47,126.0,liquidacion_tarjeta_126,3.0,478.0,tarjetas_credito_478,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN,NaT
1484,2016-09-05,-138.08,125.0,cuotas_125,3.0,127.0,otros_pasivos_127,NaN,NaN,NaN,True,False,NaN,NaN,NaN,NaN,NaT
1485,2016-09-01,-32.22,90.0,seguro_salud_90,2.0,6.0,salud_6,Sanitas S.A. de Seguros,Sanitas,Seguros de salud,True,False,NaN,NaN,NaN,NaN,NaT


In [78]:
def seguro_medico(transacciones_nomina_df):
    
    # nos quedamos con las transacciones de la categoria 'seguro_salud_90'
    transacciones_90_df=transacciones_nomina_df[transacciones_nomina_df['ID Categoría'] == 90.0]
    
    # nos quedamos con categoria desde 'Fecha transacción' hasta 'Importe'
    transacciones_90_df=transacciones_90_df.iloc[:,0:2]
    
    # renombramos columnas
    transacciones_90_df=transacciones_90_df.rename(columns={'Fecha transacción': 'FECHA', 
                                                            'Importe': 'IMPORTE'}, inplace=False)
    
    # ordenamos las fechas por orden ascendente
    transacciones_90_df=transacciones_90_df.sort_values(by=['FECHA'], ascending=True, inplace=False, 
                                                        ignore_index=True)
    
    # vamos a agrupar los valores y sumarlos por fecha para agrupar cargos distintos realizados el mismo dia
    transacciones_90_df=transacciones_90_df.groupby(['FECHA']).sum()
    
    # para que los datos sean más fáciles de interpretar vamos a hacerlos todos positivos multiplicandolos por '-1'
    transacciones_90_df['IMPORTE']=-transacciones_90_df['IMPORTE']
    
    # hacemos el dataframe del dia de pago del seguro medico
    transacciones_90_dia_df=transacciones_90_df.copy()
    
    # hacemos una columna con la fecha a partir del índice
    transacciones_90_dia_df['FECHA']=transacciones_90_dia_df.index
    
    # hacemos una columna con el dia a partir de la columna de la fecha
    transacciones_90_dia_df['DIA'] = transacciones_90_dia_df['FECHA'].dt.day
    
    # calculamos la moda - tomaremos la moda como el dia de cobro mas habitual
    stat_mode_dist = statistics.mode(transacciones_90_dia_df['DIA'])
    
    # extraemos el 1er cuartil
    quartil1_dist = int(transacciones_90_dia_df.describe().loc['25%']['DIA'])
    
    # extraemos el 3er cuartil
    quartil3_dist = int(transacciones_90_dia_df.describe().loc['75%']['DIA'])
    
    # calculamos el rango intercuartilico
    iqr_dist = quartil3_dist - quartil1_dist
    
    # si NO pasan recibo a FIN DE MES
    if stat_mode_dist < 28:    
        iqr = iqr_dist
    # si SÍ pasan recibo a FIN DE MES
    else:
        # si quartil3_dist - quartil1_dist es mayor a 4 dias
        if iqr_dist > 4:
            iqr = 4
        else:
            iqr = iqr_dist
    
    # pedimos al usuario la fecha (en produccion tomamos la fecha del sistema)
#     year = input('year: ')
#     month = input('month: ')
#     day = input('day: ')
    year = str(2019)
    month = str(1)
    day = str(20)
    
    # pasamos la fecha a string
    current_date_str= year + '-' + month + '-' + day
    
    # pasamos la fecha al formato datetime
    current_date_obj = datetime.strptime(current_date_str, '%Y-%m-%d')
    
    # pasamos iqr a formato datetime
    iqr_obj = timedelta(days=iqr)
    
    # sumamos 1 mes a current_date_obj ya que vamos a calcular fecha e importe del recibo 
    # al mes siguiente al que se lo pedimos
    target_date_obj = current_date_obj + DateOffset(months=1)
    
    # calculamos quartil3_obj en funcion de current_date_obj, quartil3_dist y si pasan el recibo a FIN de MES o NO

    # si SÍ pasan recibo a FIN de MES
    if stat_mode_dist >= 28:
        # quartil3_obj sera el ultimo dia del mes de current_date_obj
        quartil3 = calendar.monthrange(target_date_obj.year, target_date_obj.month)[1]
        quartil3_str = str(target_date_obj.year) + '-' + str(target_date_obj.month) + '-' + str(quartil3)
    # si NO pasan recibo a FIN de MES    
    else:
        quartil3_str = str(target_date_obj.year) + '-' + str(target_date_obj.month) + '-' + str(quartil3_dist) 
    
    del(target_date_obj)
    
    q3_obj = datetime.strptime(quartil3_str, '%Y-%m-%d')    
    
    q1_obj = q3_obj - iqr_obj
    
    # calculamos la moda con año, mes y dia
    if (quartil3_dist - stat_mode_dist) >= 0:
        # print('la moda esta en el mismo mes y año q quartil3_dist')
        stat_mode_str = str(q3_obj.year) + '-' + str(q3_obj.month) + '-' + str(stat_mode_dist)
    elif (stat_mode_dist - quartil1_dist) >= 0:
        # print('la moda esta en el mismo mes y año q quartil1_dist')
        stat_mode_str = str(q1_obj.year) + '-' + str(q1_obj.month) + '-' + str(stat_mode_dist)
    else:
        print('hay un fallo con el calculo de la moda')  
    
    # pasamos la moda a formato obj
    stat_mode_obj = datetime.strptime(stat_mode_str, '%Y-%m-%d')
    
    # miraremos tres meses más atras del mes objetivo para ver si hubo recibos en esos dos meses y no nos hemos 
    # dado de baja del servicio (estamos mirando desde 2 meses más atras del mes que pedimos la prediccion)
    limInfRecibo_obj = stat_mode_obj - dateutil.relativedelta.relativedelta(months=3)
    
    num_recibos = 0

    # barro desde 3 meses antes que la moda hasta el dia antes de la moda

    for d in  range(int((stat_mode_obj - limInfRecibo_obj).days)):    
        # estas lineas no se ejecutan en produccion
        ####
        # fecha_str = str((limInfRecibo_obj + timedelta(days=d)).year) + '-' + str((limInfRecibo_obj +
        #                                                                           timedelta(days=d)).month) +
        # '-' + str((limInfRecibo_obj + timedelta(days=d)).day)    
        # print(fecha_str)
        ####
        if (limInfRecibo_obj + timedelta(days=d)) in transacciones_90_dia_df.index:
            num_recibos += 1        

    # hacemos el dataframe de la serie temporal del importe de los recibos y estimados el valor del recibo
    # para el mes siguiente a la fecha en la que pedimos la prediccion
    
    transacciones_90_importe_df=transacciones_90_df.copy()
    del transacciones_90_df
    
    transacciones_90_importe_df['FECHA']=transacciones_90_importe_df.index
    
    idx = date_range(start=transacciones_90_importe_df.FECHA.min(), end=transacciones_90_importe_df.FECHA.max())
    
    # rellenamos las missing dates con NaN
    transacciones_90_importe_df = transacciones_90_importe_df.reindex(idx, fill_value='NaN')
    
    transacciones_90_importe_df.drop(columns='FECHA', inplace=True)
    
    # vamos a cambiar el tipo de 'IMPORTE' de 'object' a 'float64'    
    transacciones_90_importe_ser = transacciones_90_importe_df.T.squeeze()
    transacciones_90_importe_ser=to_numeric(transacciones_90_importe_ser, errors='coerce')
    transacciones_90_importe_df=DataFrame(transacciones_90_importe_ser)
    del transacciones_90_importe_ser
    
    # rellenamos los NaN con el valor numerico de la primera fecha anterior
    transacciones_90_importe_df['IMPORTE'].fillna(method='ffill', inplace=True)
    
    # Forecast IMPORTE With Prophet
    # Fit Prophet Model    
    df = transacciones_90_importe_df.copy()
    df['FECHA'] = df.index
    df = df[['FECHA', 'IMPORTE']]
    df.reset_index(drop=True, inplace=True)
    
    # prepare expected column names
    df.columns = ['ds', 'y']
    df['ds']= to_datetime(df['ds'])
    
    # A continuacion vamos a obtener un dataframe de train desde el 1er dia en que tenemos datos hasta el dia
    # anterior a 'current_date_str'
    limite_superior_train, = (df.index[df['ds'] == current_date_str])    
    prophet_train_df = df.iloc[:limite_superior_train,:]
    
    # define the model
    model = Prophet()
    # fit the model
    model.fit(prophet_train_df)
    
    # Make an In-Sample Forecast
    # Vamos a hacer una prediccion desde el 1er dia en que tenemos datos hasta el ultimo dia del mes siguiente
    # a current_date_str
    lim_sup_pred_obj = current_date_obj + dateutil.relativedelta.relativedelta(months=1)
    
    # calculo el ultimo dia del mes obj para hacer la prediccion del mes entero posterior a la fecha de peticion
    lim_sup_pred_str = str(lim_sup_pred_obj.year) + '-' + str(lim_sup_pred_obj.month) + '-' + \
    str(calendar.monthrange(lim_sup_pred_obj.year, lim_sup_pred_obj.month)[1])
    
    lim_sup_pred_obj = datetime.strptime(lim_sup_pred_str, '%Y-%m-%d')
    
    idx = date_range(start=transacciones_90_importe_df.index.min(), end=lim_sup_pred_obj)
    
    prophet_pred_df = DataFrame(idx)
    prophet_pred_df.columns=['ds']
    
    # use the model to make a forecast
    forecast_df = model.predict(prophet_pred_df)
    
    # esto no ira en produccion
    # plot forecast
#     model.plot(forecast_df)
#     pyplot.show()

    prediccion_recibo, = forecast_df.loc[forecast_df['ds'] == stat_mode_obj]['yhat']  
    
    # if para decidir si genero aviso utilizando la prediccion de prophet

    aviso = False
    mensaje1 = ""
    mensaje2 = ""

    if num_recibos > 0:
        mensaje1 = "Te van a pasar el próximo recibo aproximadamente el: " + stat_mode_str
        mensaje2 = "El importe aproximado del recibo será de: " + str(5*round(prediccion_recibo/5)) + ' eur'
#         print("Te van a pasar el próximo recibo aproximadamente el: " + stat_mode_str)
#         print("El importe aproximado del recibo será de: " + str(5*round(prediccion_recibo/5)) + ' eur')
        aviso = True
    else:
        aviso = False

#     print('Aviso: ' + str(aviso))
    
#     print()
#     print(transacciones_90_df)
#     print(transacciones_90_dia_df)
#     print(stat_mode_dist)
#     print(quartil1_dist)
#     print(quartil3_dist)
#     print(iqr_dist)
#     print(iqr)
#     print(current_date_str)
#     print(current_date_obj)
#     print(iqr_obj)
#     print(target_date_obj)
#     print(quartil3_str)
#     print(q3_obj)
#     print(q1_obj)
#     print(stat_mode_str)
#     print(stat_mode_obj)
#     print(limInfRecibo_obj)
#     print(num_recibos)
#     print(transacciones_90_importe_df)
#     print(idx)
#     print(transacciones_90_importe_ser)
#     print(df)
#     print(limite_superior_train)
#     print(prophet_train_df)
#     print(lim_sup_pred_obj)
#     print(lim_sup_pred_str)
#     print(prophet_pred_df)
#     print(prediccion_recibo)    
    
    return aviso, prediccion_recibo, stat_mode_str, mensaje1, mensaje2

# Load data using read_excel
transacciones_nomina_orig_df = read_excel('20210513 mmelero (249236).xlsx', sheet_name='Hoja1')

aviso_orig, prediccion_recibo_orig, stat_mode_str_orig, mensaje1_orig, \
mensaje2_orig = seguro_medico(transacciones_nomina_orig_df)

print()
print(stat_mode_str_orig)
print(prediccion_recibo_orig)
print(aviso_orig)
print(mensaje1_orig)
print(mensaje2_orig)
print('Aviso: ' + str(aviso_orig))
print()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -9.51517
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1731.18    0.00385273       45.2216           1           1      132   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     128       1733.44   0.000802763       163.675   3.864e-06       0.001      200  LS failed, Hessian reset 
     199       1736.69   0.000934381       41.9845      0.6204      0.6204      289   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     276       1738.68    0.00232856       218.263   2.164e-05       0.001      435  LS failed, Hessian reset 
     299       1738.89   0.000478353       81.0692           1           1      464   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     350       1738.97   2.80145e-05       49.1531   4.976e-07       0.001      580  LS failed, Hessian rese